In [3]:
import timm
import os
import h5py
import numpy as np
import torch
from sklearn.model_selection import KFold
import warnings
import scipy
from scipy import stats 
from scipy.stats import shapiro
warnings.filterwarnings('ignore')
import os
import matplotlib.ticker as mtick
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [69]:
#共有的区域和独有的区域
CLS2IDX =  ['emotion','gambling','language','motor','relational','social','wm']

percent = 20

all_lobe = []
# data = os.listdir('/media/D/zephyr/functional_connectivity/CHCP/aparc/percent_0.15/Emotion')
# data = [int(i[0:4]) for i in data]
# data = np.array(sorted(data))
for random_state_num in range(1998,2004):
    data = np.arange(1,1008)
    five_fold = []
    kf=KFold(n_splits=5,shuffle=True,random_state=random_state_num)
    for k,(train_index, test_index) in list(enumerate(kf.split(data))):
        all_task_all_sub = []
        for cls_num in range(6):
            one_task_all_sub = np.zeros(155)
            for sub in data[test_index]:
                sub_node = np.load('/media/D/zephyr/vit_155_155/other_baseline/vis/{}/{}_{}_node_{}_6class_original.npy'
                .format(percent,CLS2IDX[cls_num],sub,random_state_num))
                # sub_node = (sub_node - np.mean(sub_node)) / np.std(sub_node)
                # one_task_all_sub = one_task_all_sub +sub_node
                one_task_all_sub = one_task_all_sub + (sub_node-sub_node.min())/(sub_node.max()-sub_node.min())
            all_task_all_sub.append((one_task_all_sub-one_task_all_sub.min())/(one_task_all_sub.max()-one_task_all_sub.min()))
        five_fold.append(all_task_all_sub)
    all_lobe.append(five_fold)

In [70]:
six_lobe = np.load('/media/D/zephyr/vit_155_155/tulunzhibiao/bigger_six_region_of_155_region.npy')
all_data = []
index_all = []
for lobes in np.arange(1,6):
    sulci_indx = [i for i in np.where(six_lobe==lobes)[0] if i%2==0]
    gyri_indx = [i for i in np.where(six_lobe==lobes)[0] if i%2==1]
    index_all = index_all + sulci_indx+gyri_indx

In [71]:
for task_num in range(6):
    mean_value =np.mean(np.array(all_lobe).transpose(2,3,0,1)[task_num].reshape(155,-1),1)[index_all+list(np.arange(136,155))]
    std_value = stats.sem(np.array(all_lobe).transpose(2,3,0,1)[task_num].reshape(155,-1),1)[index_all+list(np.arange(136,155))]
    min_95 = [ stats.norm.interval(alpha=0.95, 
                    loc=mean_value[i],
                    scale=std_value[i])[0] for i in np.arange(155)]
    max_95 = [ stats.norm.interval(alpha=0.95, 
                    loc=mean_value[i],
                    scale=std_value[i])[1] for i in np.arange(155)]
    a = pd.read_excel('/media/D/zephyr/vit_155_155/code_for_anlysis/uestc/region_new.xlsx')
    a.insert(loc=2, column='value', value=mean_value)
    a.insert(loc=3, column='sd', value=std_value)
    a.insert(loc=4, column='min95', value=min_95)
    a.insert(loc=5, column='max95', value=max_95)
    a.to_excel('/media/D/zephyr/vit_155_155/picture_new/hcp/aparc/important_lobes/6class_hcp_dk_{}.xlsx'.format(CLS2IDX[task_num]))

In [ ]:
# spearman corr between two atlas
HcpSchaeferValue = np.mean(np.array(all_lobe).transpose(2,3,0,1).reshape(7,219,-1),2)
HcpDkValue = np.mean(np.array(all_lobe).transpose(2,3,0,1).reshape(7,155,-1),2)
Sch_all = []
for task in range(7):
    six_lobe = np.load('/media/D/zephyr/vit_155_155/other_map/scheafer_lobe.npy')
    Schaefer_value_all = []
    for lobes in np.arange(1,6):
        sulci_indx = [i for i in np.where(six_lobe==lobes)[0]]
        # sulci_indx = [i for i in np.where(six_lobe==lobes)[0] if i%2==0]
        Schaefer_value_all.append(np.mean(HcpSchaeferValue[task][sulci_indx]))
        # gyri_indx = [i for i in np.where(six_lobe==lobes)[0] if i%2==1]
        # Schaefer_value_all.append(np.mean(HcpSchaeferValue[task][gyri_indx]))
    Schaefer_value_all.append(np.mean(HcpSchaeferValue[task][list(np.arange(200,219))]))
    Sch_all.append(Schaefer_value_all)

Dk_all = []    
for task in range(7):
    six_lobe = np.load('/media/D/zephyr/vit_155_155/tulunzhibiao/bigger_six_region_of_155_region.npy')
    DK_index_all = []
    for lobes in np.arange(1,6):
        sulci_indx = [i for i in np.where(six_lobe==lobes)[0] ]
        # sulci_indx = [i for i in np.where(six_lobe==lobes)[0] if i%2==0]
        DK_index_all.append(np.mean(HcpDkValue[task][sulci_indx]))
        # gyri_indx = [i for i in np.where(six_lobe==lobes)[0] if i%2==1]
        # DK_index_all.append(np.mean(HcpDkValue[task][gyri_indx]))
    DK_index_all.append(np.mean(HcpDkValue[task][list(np.arange(136,155))]))
    Dk_all.append(DK_index_all)

for task in range(7):
    DK_data = Dk_all[task]
    Sch_data = Sch_all[task]
    print(scipy.stats.spearmanr(DK_data, Sch_data))

In [ ]:
# spearman corr between two datasets
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter    
CLS2IDX =  ['emotion','gambling','language','motor','relational','social','wm']
for task in range(7):
    hcp_data = pd.read_excel('/media/D/zephyr/vit_155_155/picture_new/hcp/aparc/important_lobes/hcp_dk_{}.xlsx'.format(CLS2IDX[task]))
    chcp_data = pd.read_excel('/media/D/zephyr/vit_155_155/picture_new/hcp/aparc/important_lobes/chcp_dk_{}.xlsx'.format(CLS2IDX[task]))
    column = ['DK_value']
    df = pd.DataFrame(columns=column,data = np.array(hcp_data['value']))
    df.insert(loc=1, column='SCH_value', value= chcp_data['value'])
    sns.set_style("whitegrid", {'axes.grid' : False})
    g = sns.lmplot(x="DK_value",y="SCH_value",data = df,size = 5,scatter_kws = dict(s = 100, linewidths = 0.8, edgecolors = 'black') )
    plt.ylabel('',fontdict={'family' : 'Arial', 'size'   : 30})
    plt.xlabel('',fontdict={'family' : 'Arial', 'size'   : 30})
    ax = g.axes
    ax[0, 0].spines['bottom'].set_linewidth(2)
    ax[0, 0].spines['left'].set_linewidth(2)
    ax[0, 0].spines['top'].set_visible(False)
    ax[0, 0].spines['right'].set_visible(False)
    ax[0, 0].spines['left'].set_color('black')
    ax[0, 0].spines['bottom'].set_color('black')
    plt.yticks([])
    plt.xticks([])
    plt.savefig("/media/D/zephyr/vit_155_155/picture_new/scatter/dk_{}.svg".format(CLS2IDX[task]), dpi=300,format="svg")

CLS2IDX =  ['emotion','gambling','language','motor','relational','social','wm']
for task in range(7):
    hcp_data = pd.read_excel('/media/D/zephyr/vit_155_155/picture_new/hcp/aparc/important_lobes/schaefer_hcp_{}.xlsx'.format(CLS2IDX[task]))
    chcp_data = pd.read_excel('/media/D/zephyr/vit_155_155/picture_new/hcp/aparc/important_lobes/schaefer_chcp_{}.xlsx'.format(CLS2IDX[task]))
    print(scipy.stats.spearmanr(hcp_data['value'], chcp_data['value']))